In [1]:
import tensorflow as tf
import mnist
import os

# Defining sizes

In [14]:
num_classes = 10
img_rows, img_cols = 28, 28
num_channels = 1
input_shape = (img_rows, img_cols, num_channels)

# Loading Data

In [16]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Defining Model

In [19]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='selu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

writer = tf.summary.create_file_writer('./model_logs')
with writer.as_default():
    tf.summary.scalar('custom_log', 10, step=3)

In [18]:
model.compile(optimizer='sgd',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
callbacks = [tf.keras.callbacks.TensorBoard('./logs_keras',profile_batch = 100000000)]

model.fit(x_train, y_train, epochs=5, verbose=1, validation_data=(x_test, y_test), callbacks=callbacks)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 92us/sample - loss: 0.5292 - accuracy: 0.8596 - val_loss: 0.3406 - val_accuracy: 0.9058
Epoch 2/5
60000/60000 [==============================] - 5s 83us/sample - loss: 0.3335 - accuracy: 0.9058 - val_loss: 0.3010 - val_accuracy: 0.9164
Epoch 3/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.3022 - accuracy: 0.9145 - val_loss: 0.2808 - val_accuracy: 0.9224
Epoch 4/5
60000/60000 [==============================] - 6s 102us/sample - loss: 0.2834 - accuracy: 0.9201 - val_loss: 0.2677 - val_accuracy: 0.9251
Epoch 5/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.2685 - accuracy: 0.9248 - val_loss: 0.2562 - val_accuracy: 0.9270


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100480    
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [9]:
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(128, activation='sigmoid'))
model1.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [10]:
model1.compile(optimizer='sgd',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model1.fit(x_train, y_train, epochs=5, verbose=1, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 91us/sample - loss: 1.4186 - accuracy: 0.7014 - val_loss: 0.8498 - val_accuracy: 0.8375
Epoch 2/5
60000/60000 [==============================] - 5s 91us/sample - loss: 0.6925 - accuracy: 0.8508 - val_loss: 0.5555 - val_accuracy: 0.8746
Epoch 3/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.5178 - accuracy: 0.8746 - val_loss: 0.4541 - val_accuracy: 0.8889
Epoch 4/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.4445 - accuracy: 0.8851 - val_loss: 0.4024 - val_accuracy: 0.8965
Epoch 5/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.4038 - accuracy: 0.8920 - val_loss: 0.3732 - val_accuracy: 0.8998


# Lazy vs Eager execution

In [11]:
a = tf.constant([1, 2, 3])
b = tf.constant([0, 0, 1])
c = tf.add(a, b)

print(c)
print(c.numpy())

tf.Tensor([1 2 4], shape=(3,), dtype=int32)
[1 2 4]


# Creating graphs at TensorFlow 2

In [12]:
def compute(a, b, c):
    d = a * b + c
    e = a * b * c
    return d, e

# Tensorflow AutoGraph and tf.function

In [13]:
@tf.function
def compute(a, b, c):
    d = a * b + c
    e = a * b * c
    return d, e

# Backpropagating errors using the gradient tape

In [14]:
A, B =  tf.constant(3.0), tf.constant(6.0)
X = tf.Variable(20.0) # In practice we would start with a random value
loss = tf.math.abs(A * X - B)

In [15]:
loss

<tf.Tensor: id=67061, shape=(), dtype=float32, numpy=54.0>

In [16]:
def train_step():
    with tf.GradientTape() as tape:
        loss = tf.math.abs(A * X - B)
    dX = tape.gradient(loss, X)
    
    print('X = {:.2f}, dX = {:2f}'.format(X.numpy(), dX))
    X.assign(X - dX)
    
for i in range(8):
    train_step()

X = 20.00, dX = 3.000000
X = 17.00, dX = 3.000000
X = 14.00, dX = 3.000000
X = 11.00, dX = 3.000000
X = 8.00, dX = 3.000000
X = 5.00, dX = 3.000000
X = 2.00, dX = 0.000000
X = 2.00, dX = 0.000000


# Functional API for model

In [17]:
model_input = tf.keras.layers.Input(shape=input_shape)
output = tf.keras.layers.Flatten()(model_input)
output = tf.keras.layers.Dense(128, activation='relu')(output)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(output)
model2 = tf.keras.Model(model_input, output)

# Training a custom Estimator

In [18]:
estimator = tf.keras.estimator.model_to_estimator(model, model_dir='.')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Note that this doesn't affect the state of the model instance you passed as `keras_model` argument.
INFO:tensorflow:Using config: {'_model_dir': '.', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000024A8A244CC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_m

In [19]:
BATCH_SIZE = 32
def train_input_fn():
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.batch(BATCH_SIZE).repeat()
    return train_dataset

In [ ]:
estimator.train(train_input_fn, steps=len(x_train)//BATCH_SIZE)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [12]:
%load_ext tensorboard
%tensorboard --logdir logs_keras


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.